In [7]:
import plotly.express as px
import numpy as np
import pandas as pd

Import the datasets from data/

In [13]:
wi_grid = pd.read_csv("../data/mygrid.csv")
r0_county = pd.read_csv("../data/df_gg_ms.csv")

In [17]:
r0_county_grid = pd.merge(r0_county, wi_grid, how = "left", on = "code")

In [18]:
r0_county_grid

,county,t_start,t_end,Mean(R),Std(R),Quantile.0.025(R),Quantile.0.05(R),Quantile.0.25(R),Median(R),Quantile.0.75(R),Quantile.0.95(R),Quantile.0.975(R),Date,code,Risk,yintercept,name,row,col
0,Adams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adams,Low-Case Counts,NaN,Adams County,6,6
1,Ashland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ashland,Low-Case Counts,NaN,Ashland County,2,4
2,Barron,2.0,8.0,3.498576,1.428288,1.283916,1.523639,6.130109,3.306249,6.130109,6.130109,6.803759,1.586477e+12,barron,Low,NaN,Barron County,3,2
3,Barron,3.0,9.0,2.684234,1.095834,0.985067,1.168990,4.703241,2.536673,4.703241,4.703241,5.220089,1.586563e+12,barron,Low,NaN,Barron County,3,2
4,Barron,4.0,10.0,1.090765,0.629754,0.224942,0.297303,2.289078,0.972257,2.289078,2.289078,2.626813,1.586650e+12,barron,Low,NaN,Barron County,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2507,Winnebago,59.0,65.0,1.005305,0.197156,0.656698,0.704430,1.350050,0.992446,1.350050,1.350050,1.426950,1.589674e+12,winnebago,High,NaN,Winnebago County,6,8
2508,Winnebago,60.0,66.0,1.013614,0.202723,0.655957,0.704750,1.368476,1.000131,1.368476,1.368476,1.447850,1.589760e+12,winnebago,High,NaN,Winnebago County,6,8
2509,Winnebago,61.0,67.0,1.098441,0.215422,0.717538,0.769692,1.475125,1.084391,1.475125,1.475125,1.559149,1.589846e+12,winnebago,High,NaN,Winnebago County,6,8
2510,Winnebago,62.0,68.0,1.431578,0.249206,0.985432,1.047771,1.864628,1.417143,1.864628,1.864628,1.959719,1.589933e+12,winnebago,High,1.0,Winnebago County,6,8


In [33]:
rows = sorted(set(wi_grid["row"].tolist()))
cols = sorted(set(wi_grid["col"].tolist()))

## Attempt 1: using plotly express
Problem is I don't know how to control the individual titles

In [43]:
fig = px.line(r0_county_grid, x="t_end", y="Mean(R)", facet_col="col", facet_row="row",
category_orders={"col": cols, "row": rows})
fig.for_each_annotation(lambda a: a.update(text=""))
fig.for
fig.show()

## Attempt 2
Most customized approch, but need to fix a lot of things.  I think this is a good starting point

In [140]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import itertools

x = np.array([(x, y) for x in rows for y in cols])
x = pd.DataFrame([(x, y) for x in rows for y in cols], columns=["row","col"])

x = pd.merge(x, wi_grid, how = "left", on = ["row","col"])
x = x.fillna("")


# create figure with subplots
fig = make_subplots(rows=max(rows), cols=max(cols), subplot_titles = [y.replace(" County", "") for y in x["name"].tolist()])
fig.for_each_annotation(lambda a: a.update(font=dict(size = 7)))

for row, col in itertools.product(rows, cols):
    index = (r0_county_grid["row"] == row) & (r0_county_grid["col"] == col)
    subset_df = r0_county_grid[index]
    if not subset_df.empty:
        fig.add_trace(
            go.Scatter(x=subset_df["t_end"], y=subset_df["Mean(R)"], mode = "lines"),
            row = row,
            col = col
        )
    

# fig.add_trace(
#     go.Scatter(x=[1, 2, 3], y=[4, 5, 6]),
#     row=1, col=1
# )

# fig.add_trace(
#     go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),
#     row=1, col=2
# )

fig.show()

## Everything below this line is scratch work!

In [130]:
index = (r0_county_grid["row"] == 1) & (r0_county_grid["col"] == 1)
subset_df = r0_county_grid[index]
subset_df.empty
#     

# r0_county_grid[]

True

In [4]:
fig = px.scatter(px.data.tips(), x="total_bill", y="tip", facet_col="smoker")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [8]:
print(px.data.tips())

total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]


In [53]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# create figure with subplots
fig = make_subplots(rows=2, cols=2, subplot_titles = ['title1','title2',"title3"])

fig.add_trace(
    go.Scatter(x=[1, 2, 3], y=[4, 5, 6]),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),
    row=1, col=2
)

fig.update_layout(height=600, width=800, title_text="Subplots")

fig.show()

# to change subtitle, address subplot
fig['layout']['annotations'][0].update(text='your text here');
print(fig["layout"]["annotations"])

fig.show()


# fig.add_annotation(text = "title4", row = 2, col = 2)
# fig.show()

(layout.Annotation({
    'font': {'size': 16},
    'showarrow': False,
    'text': 'your text here',
    'x': 0.225,
    'xanchor': 'center',
    'xref': 'paper',
    'y': 1.0,
    'yanchor': 'bottom',
    'yref': 'paper'
}), layout.Annotation({
    'font': {'size': 16},
    'showarrow': False,
    'text': 'title2',
    'x': 0.775,
    'xanchor': 'center',
    'xref': 'paper',
    'y': 1.0,
    'yanchor': 'bottom',
    'yref': 'paper'
}), layout.Annotation({
    'font': {'size': 16},
    'showarrow': False,
    'text': 'title3',
    'x': 0.225,
    'xanchor': 'center',
    'xref': 'paper',
    'y': 0.375,
    'yanchor': 'bottom',
    'yref': 'paper'
}))
